In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter

def get_device():
    """
    Get the device to run the model on (GPU if available, else CPU).
    
    Returns:
        device (torch.device): The device to be used.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    return device

def load_data(data_dir, batch_size, img_size):
    """
    Load and preprocess the dataset.
    
    Args:
        data_dir (str): The directory containing the dataset.
        batch_size (int): The batch size for data loading.
        img_size (int): The size to resize images to.
        
    Returns:
        train_loader (DataLoader): The DataLoader for the training set.
        val_loader (DataLoader): The DataLoader for the validation set.
        test_loader (DataLoader): The DataLoader for the test set.
        num_classes (int): The number of classes in the dataset.
    """
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    num_classes = len(dataset.classes)
    
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    train_data, val_data, test_data = random_split(dataset, [train_size, val_size, test_size])
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader, test_loader, num_classes

def build_model(num_classes):
    """
    Build a ResNet50 model with a custom fully connected layer.
    
    Args:
        num_classes (int): The number of output classes.
        
    Returns:
        model (nn.Module): The ResNet50 model with a modified fully connected layer.
    """
    model = models.resnet50(pretrained=True)
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes)
    )
    return model

def train_and_evaluate(model, train_loader, val_loader, device, epochs, model_path, optimizer_type='Adam', learning_rate=1e-3, use_tensorboard=True):
    """
    Train and evaluate the model.
    
    Args:
        model (nn.Module): The model to be trained.
        train_loader (DataLoader): The DataLoader for the training set.
        val_loader (DataLoader): The DataLoader for the validation set.
        device (torch.device): The device to run the model on.
        epochs (int): The number of epochs to train for.
        model_path (str): The path to save the best model.
        optimizer_type (str, optional): The type of optimizer to use. Defaults to 'Adam'.
        learning_rate (float, optional): The learning rate for the optimizer. Defaults to 1e-3.
        use_tensorboard (bool, optional): Whether to use TensorBoard for visualization. Defaults to True.
    """
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    if optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    else:
        raise ValueError("Unsupported optimizer type. Please choose 'Adam' or 'SGD'.")
    
    best_val_loss = float('inf')
    
    if use_tensorboard:
        writer = SummaryWriter()
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        all_preds, all_labels = [], []
        correct = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        
        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / len(train_loader.dataset)
        
        model.eval()
        val_loss = 0.0
        val_correct = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_correct += (predicted == labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = 100 * val_correct / len(val_loader.dataset)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_path)
            print(f"Model saved at epoch {epoch+1}")
        
        print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}% | Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%")
        
        cm = confusion_matrix(all_labels, all_preds)
        disp = ConfusionMatrixDisplay(cm)
        disp.plot(cmap='Blues')
        plt.show()
        
        if use_tensorboard:
            writer.add_scalar('Loss/train', train_loss, epoch)
            writer.add_scalar('Loss/val', val_loss, epoch)
            writer.add_scalar('Accuracy/train', train_accuracy, epoch)
            writer.add_scalar('Accuracy/val', val_accuracy, epoch)
            writer.add_figure('Confusion Matrix', disp.figure_, epoch)
    
    if use_tensorboard:
        writer.close()

def evaluate_on_test_set(model, test_loader, device):
    """
    Evaluate the model on the test set.
    
    Args:
        model (nn.Module): The model to be evaluated.
        test_loader (DataLoader): The DataLoader for the test set.
        device (torch.device): The device to run the model on.
    """
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_loss /= len(test_loader)
    accuracy = 100 * correct / total
    print(f"Test Loss: {test_loss:.4f} | Accuracy: {accuracy:.2f}%")

def main():
    data_dir = "../hand_gesture_recognition_project/leapGestRecog"
    batch_size = 60
    img_size = 224
    epochs = 50
    model_path = "best_model.pth"
    optimizer_type = 'Adam'
    learning_rate = 1e-3
    use_tensorboard = True
    
    device = get_device()
    train_loader, val_loader, test_loader, num_classes = load_data(data_dir, batch_size, img_size)
    model = build_model(num_classes)
    train_and_evaluate(model, train_loader, val_loader, device, epochs, model_path, optimizer_type, learning_rate, use_tensorboard)
    evaluate_on_test_set(model, test_loader, device)

if __name__ == "__main__":
    main()
